In [54]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk
from nltk.corpus import stopwords

from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer

%matplotlib inline

In [2]:
# nltk.download()  # 1 fois seulement

In [68]:
df = pd.read_csv("temp.csv", encoding = "ISO-8859-1")
tags = pd.read_csv("top104.csv")

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9348 entries, 0 to 9347
Data columns (total 3 columns):
Title    9348 non-null object
Tags     9348 non-null object
Body     9348 non-null object
dtypes: object(3)
memory usage: 219.2+ KB


## Title

## Tags

In [5]:
taglist = tags["TagName"].tolist()

In [6]:
labels = df["Tags"].str.replace("><", "*").str.replace("[><]*", "").str.split("*").tolist()

In [7]:
counter_class = Counter()
for post in labels:
    for label in post:
        counter_class[label] += 1
        
len(counter_class)

5405

In [8]:
empty = 0
empty_counter = Counter()
added = []
index = []
for i, tl in enumerate(labels):
    for key in tl:
        if key not in taglist:
            empty_counter[key] += 1

for key, freq in empty_counter.items():
    if freq > 25:
        added.append(key)

taglist += added
for i, tl in enumerate(labels):
    labels[i] = [word for word in tl if word in taglist]
    if len(labels[i]) == 0:
        empty += 1
        index.append(i)
        for tag in tl:
            empty_counter[tag] += 1
print(empty)

0


In [9]:
counter_class = Counter()
for post in labels:
    for label in post:
        counter_class[label] += 1
        
len(counter_class)

194

In [127]:
labels

[['c++', 'language-lawyer', 'c++1z'],
 ['javascript', 'reactjs'],
 ['c++', 'c++11', 'c++14', 'language-lawyer', 'c++1z'],
 ['mysql', 'sql'],
 ['scala'],
 ['javascript', 'jquery', 'html', 'image'],
 ['python'],
 ['c++', 'c++11', 'stl', 'language-lawyer'],
 ['java', 'string', 'lambda', 'java-stream'],
 ['c++', 'templates', 'lambda', 'language-lawyer', 'c++1z'],
 ['git'],
 ['angular', 'typescript'],
 ['java', 'scala'],
 ['r', 'regex'],
 ['python', 'python-3.x', 'list'],
 ['scala', 'apache-spark'],
 ['r'],
 ['python', 'recursion'],
 ['python', 'sqlite'],
 ['c'],
 ['php'],
 ['c'],
 ['python', 'dictionary'],
 ['c++', 'c++11', 'c++1z'],
 ['c++', 'c', 'gcc'],
 ['python', 'algorithm', 'sorting'],
 ['python', 'python-3.x'],
 ['javascript', 'jquery', 'arrays'],
 ['multithreading', 'f#'],
 ['haskell'],
 ['python', 'string'],
 ['javascript', 'angular'],
 ['vba', 'excel-vba'],
 ['java'],
 ['c'],
 ['c++', 'enums'],
 ['ajax'],
 ['java', 'android', 'generics', 'inheritance'],
 ['javascript', 'html5'],


In [130]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(labels)

In [131]:
y.shape

(9348, 194)

In [98]:
# df2 = df.drop(df.index[index]).reset_index(drop=True)

In [105]:
# df2.to_csv("temp.csv", index=False)

## Body

## Pre-cleanup

In [70]:
df["Body"] = df["Title"] + " " + df['Body']

In [71]:
df["Body"] = df["Body"].str.replace("<code>[^<]*</code>", "")

In [72]:
df["Body"] = df["Body"].str.replace("<[^>]*>", "")

In [99]:
df["Body"] = df["Body"].str.replace("\r", "").str.replace("\n", "")

In [100]:
df["Body"].iloc[30]

"Python fstring as function I'd like to use Python f-string for its syntactical simplicity, compared to string.Template() or other approach. However, in my application, the string is loaded from file, and the values of the variable can only be provided later.If there a way to invoke fstring functionality separate from the string definition? Hopefully code below will better explain what I hope to achieve."

In [101]:
corpus = df["Body"].tolist()

In [102]:
corpus

['Anonymous temporaries and class template argument deduction - gcc vs clang Consider the following code snippet:g++ 7 happily creates a temporary object of type , deducing .clang++ 5 and 6 refuse to compile the code:live example on wandboxIs this a clang bug, or is there something in the Standard that prevents class template argument deduction from kicking in for unnamed temporaries?',
 "Is there a reason people prefer using the constructor of a React component instead of componentWillMount? I find that using the lifecycle method  to setup initial state......is slightly simpler than using the constructor. Namely because when you use the constructor you have to call .Not only that but if your component is passed  and/or  then you have to manually pass those through as well.I haven't had any problems using  but I virtually never see anyone using it for setting up state (unless they are avoiding es6 and don't have classes). I get that we have access to the constructor in an es6 class, bu

## Increase Stop_word

In [116]:
stemmer = nltk.stem.PorterStemmer()
tokenizer = nltk.RegexpTokenizer(r'(\w+|\d+)')

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = tokenizer.tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

In [117]:
count = Counter()
tokenizer = nltk.RegexpTokenizer(r'\w+')

for i, sentence in enumerate(corpus):
    tokenized = tokenizer.tokenize(sentence)
    for word in tokenized:
        count[word] += 1

stop = set(stopwords.words('english'))# | set([word for word, freq in count.most_common(100)])

In [118]:
tfidf = TfidfVectorizer(min_df = 0.001, tokenizer=tokenize, stop_words=stop)
tfs = tfidf.fit_transform(corpus)

In [119]:
tfs

<9348x2944 sparse matrix of type '<class 'numpy.float64'>'
	with 394494 stored elements in Compressed Sparse Row format>

In [120]:
tfidf.vocabulary_

{'anonym': 214,
 'temporari': 2578,
 'class': 490,
 'templat': 2577,
 'argument': 248,
 'deduct': 705,
 'gcc': 1120,
 'vs': 2833,
 'clang': 486,
 'consid': 583,
 'follow': 1055,
 'code': 519,
 'snippet': 2396,
 'g': 1108,
 '7': 103,
 'happili': 1201,
 'creat': 641,
 'object': 1765,
 'type': 2696,
 'deduc': 704,
 '5': 87,
 '6': 96,
 'refus': 2137,
 'compil': 549,
 'live': 1525,
 'exampl': 936,
 'thi': 2601,
 'bug': 403,
 'someth': 2409,
 'standard': 2450,
 'prevent': 1991,
 'unnam': 2744,
 'reason': 2109,
 'peopl': 1890,
 'prefer': 1981,
 'use': 2771,
 'constructor': 594,
 'react': 2095,
 'compon': 555,
 'instead': 1351,
 'find': 1022,
 'lifecycl': 1507,
 'method': 1627,
 'setup': 2323,
 'initi': 1332,
 'state': 2454,
 'slightli': 2383,
 'simpler': 2362,
 'name': 1701,
 'becaus': 324,
 'call': 422,
 'onli': 1793,
 'pass': 1876,
 'manual': 1580,
 'well': 2864,
 'ani': 210,
 'problem': 2007,
 'virtual': 2822,
 'never': 1723,
 'see': 2289,
 'anyon': 219,
 'set': 2321,
 'unless': 2741,
 'av

# classifier MultiLable Classifier

In [135]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=10, random_state=0)
clf.fit(tfs, y)

[ 0.00085258  0.          0.         ...,  0.          0.          0.0001415 ]
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


In [136]:
print(clf.feature_importances_.shape)

(2944,)


In [157]:
pred = clf.predict(tfs[:5])
pred
# ix = np.argwhere(pred==1)
# print(ix)

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.

In [155]:
tfs[:5]

<5x2944 sparse matrix of type '<class 'numpy.float64'>'
	with 179 stored elements in Compressed Sparse Row format>

# autre

In [123]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [125]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF

tfidf_feature_names = tfidf.get_feature_names()

no_topics = 194

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfs)

no_top_words = 10
display_topics(nmf, tfidf_feature_names, no_top_words)

Topic 0:
zoom extra factor fact facebook face fabric f eye extrem
Topic 1:
zoom extra factor fact facebook face fabric f eye extrem
Topic 2:
zoom extra factor fact facebook face fabric f eye extrem
Topic 3:
zoom extra factor fact facebook face fabric f eye extrem
Topic 4:
file folder read line open name directori download pdf write
Topic 5:
array sort numpi index want two like pointer size 2d
Topic 6:
zoom extra factor fact facebook face fabric f eye extrem
Topic 7:
zoom extra factor fact facebook face fabric f eye extrem
Topic 8:
list item sort want comprehens convert tupl haskel dict two
Topic 9:
gener random pdf html interfac rust entiti apk kotlin differ
Topic 10:
function call pass argument haskel member defin callback paramet write
Topic 11:
io 11 10 devic iphon bar simul appl navig crash
Topic 12:
modul 9 packag depend import requir export jar maven angular
Topic 13:
android studio com 3 devic gradl googl 0 sdk 26
Topic 14:
zoom extra factor fact facebook face fabric f eye extre